In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import base64
%matplotlib inline

In [ ]:
authurl = 'https://accounts.spotify.com/api/token'
authheader = {}
authdata = {}

In [ ]:
with open('../keys/client_id.json') as fi:
    credentials = json.load(fi)

In [ ]:
client_id = credentials['client_id']

In [ ]:
with open('../keys/client_secret_id.json') as fi:
    secret_credentials = json.load(fi)

In [ ]:
client_secret_id = secret_credentials['client_secret_id']

In [ ]:
def getaccesstoken(client_id, client_sercret_id):
    message = f'{client_id}:{client_secret_id}'
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii')
    
    authheader['Authorization'] = 'Basic ' + base64_message
    authdata['grant_type'] = 'client_credentials'
    
    res = requests.post(authurl, headers = authheader, data = authdata)
    
    responseobject = res.json()
    
    access_token = responseobject['access_token']
    
    return access_token

In [ ]:
token = getaccesstoken(client_id, client_secret_id)

In [ ]:
albumid = '1H81jGoWeLI8ufq42GfDPn'

In [ ]:
def get_album_tracks(token, albumid):
    
    album_endpoint = f'https://api.spotify.com/v1/albums/{albumid}'
    
    getheader = {
        'Authorization': 'Bearer ' + token
    }
    
    res = requests.get(album_endpoint, headers=getheader)
    
    album_object = res.json()
    
    return album_object

In [ ]:
album_list = get_album_tracks(token, albumid)

In [ ]:
print(json.dumps(album_list, indent=2))